<img src="./images/caching.png" width="800">

# Caching in Python

Caching is a powerful technique that allows you to store the results of expensive function calls and return the cached result when the same inputs occur again. This can significantly improve the efficiency of your programs by avoiding redundant computations. In this lecture, we will explore different caching strategies provided by Python's `functools` module, such as `@cache` and `@lru_cache`, and discuss how to implement a cache with a time-to-live (TTL) feature.


**Table of contents**<a id='toc0_'></a>    
- [Understanding Caching](#toc1_)    
  - [The Basics of Caching](#toc1_1_)    
- [Using `@cache`](#toc2_)    
  - [Example of `@cache`](#toc2_1_)    
- [Using `@lru_cache`](#toc3_)    
  - [Configuring `@lru_cache`](#toc3_1_)    
- [Implementing Cache with TTL](#toc4_)    
  - [Using `cachetools` for TTL](#toc4_1_)    
  - [Building a Custom TTL Cache](#toc4_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>[Understanding Caching](#toc0_)


Caching can be particularly beneficial in scenarios where a function is deterministic (the same inputs always produce the same output) and the function is called repeatedly with the same arguments. By storing the results of these function calls, we can reduce the computational overhead and speed up our programs.


### <a id='toc1_1_'></a>[The Basics of Caching](#toc0_)


At its core, caching involves storing the results of a function call along with its arguments. When the function is called, the cache is checked to see if the result for those arguments is already available. If it is, the cached result is returned immediately, skipping the actual computation.


## <a id='toc2_'></a>[Using `@cache`](#toc0_)


Introduced in Python 3.9, the `@cache` decorator is a simple way to add caching to a function without any configuration.


### <a id='toc2_1_'></a>[Example of `@cache`](#toc0_)


In [1]:
from functools import cache

@cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

In [2]:
# The first call will compute the result
fibonacci(10)

55

In [3]:
# Subsequent calls with the same argument will fetch the result from the cache
fibonacci(10)

55

## <a id='toc3_'></a>[Using `@lru_cache`](#toc0_)


For more control over the caching behavior, `@lru_cache` can be used. It implements a least-recently-used cache, which evicts the least recently accessed items when the cache is full.


### <a id='toc3_1_'></a>[Configuring `@lru_cache`](#toc0_)


You can specify the maximum size of the cache using the `maxsize` argument. Additionally, you can obtain cache statistics and clear the cache when needed.


In [4]:
from functools import lru_cache

In [5]:
@lru_cache(maxsize=128)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)


In [6]:
# The cache_info method provides statistics about cache usage
fibonacci.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

In [7]:
fibonacci(100)

354224848179261915075

In [8]:
fibonacci.cache_info()

CacheInfo(hits=98, misses=101, maxsize=128, currsize=101)

In [9]:
# The cache can be cleared with the cache_clear method
fibonacci.cache_clear()

In [10]:
fibonacci.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

The `cache_info()` method of a cached function using `@lru_cache` from Python's `functools` module returns a named tuple, `CacheInfo`, containing statistics about the performance of the cache. Here's what each of the fields in `CacheInfo(hits=9, misses=11, maxsize=None, currsize=11)` as an example represent:

1. `hits`: The number of times a cache hit occurred. A cache hit is when the requested data is found in the cache, meaning the function does not need to execute because the result is already stored and can be returned immediately. In this case, there have been 9 cache hits.

2. `misses`: The number of times a cache miss occurred. A cache miss is when the requested data is not found in the cache, requiring the function to execute and compute the result, which is then stored in the cache for future use. In this case, there have been 11 cache misses.

3. `maxsize`: The maximum size of the cache, as defined by the `maxsize` parameter of the `@lru_cache` decorator. If `maxsize` is set to `None`, the cache can grow without bound. In this case, `maxsize` is `None`, meaning there is no fixed limit to the number of entries that the cache can hold.

4. `currsize`: The current size of the cache, i.e., the number of entries currently stored in the cache. This number will always be less than or equal to `maxsize` if a `maxsize` is set. In this case, the current size of the cache is 11, which means there are 11 unique entries that have been stored.

The `cache_info` statistics can be useful to understand and optimize the performance of the caching mechanism. High numbers of hits indicate that the cache is being effectively used, while a high number of misses may suggest that the cache size (`maxsize`) could be increased (if not `None`), or that the function's inputs are too varied and unique for the cache to be effective.

## <a id='toc4_'></a>[Implementing Cache with TTL](#toc0_)


Sometimes, you may want cached results to expire after a certain period. This is not directly supported by `@cache` or `@lru_cache`, but you can use third-party libraries or implement your own TTL cache.


### <a id='toc4_1_'></a>[Using `cachetools` for TTL](#toc0_)


The `cachetools` library provides a variety of cache implementations, including TTLCache, which supports time-to-live functionality.


In [11]:
%pip install cachetools

from cachetools import cached, TTLCache
import time

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Create a TTLCache instance with a TTL of 300 seconds
ttl_cache = TTLCache(maxsize=100, ttl=300)

In [13]:
@cached(ttl_cache)
def get_data(key):
    # Simulate a time-consuming data retrieval process
    time.sleep(2)
    return f"Data for {key}"

In [14]:
# Use the cache as usual, knowing that entries will expire after the TTL
get_data('key1')

'Data for key1'

In [15]:
get_data('key1')  # This call will retrieve the result from the cache

'Data for key1'

### <a id='toc4_2_'></a>[Building a Custom TTL Cache](#toc0_)


For a more hands-on approach, you can create a decorator that adds TTL to a simple caching mechanism.


In [16]:
import time
import functools

In [17]:
def ttl_cache(ttl: int):
    def decorator(func):
        cache = {}
        @functools.wraps(func)
        def wrapped(*args, **kwargs):
            nonlocal cache
            current_time = time.time()
            cache_key = args + tuple(sorted(kwargs.items()))

            # Check cache for existing result and validity
            if cache_key in cache:
                result, timestamp = cache[cache_key]
                if current_time - timestamp < ttl:
                    return result

            # Compute and cache new result
            result = func(*args, **kwargs)
            cache[cache_key] = (result, current_time)
            return result
        return wrapped
    return decorator

In [18]:
# Usage example
@ttl_cache(ttl=300)
def get_data(key):
    # Time-consuming computation
    return f"Data for {key}"

In [19]:
# The cached result will expire after 300 seconds
get_data('key1')


'Data for key1'

In this lecture, we have explored various caching strategies and their implementations in Python. Caching is an essential tool in optimizing performance, and understanding these techniques will help you write more efficient and faster-running programs.